В этом ноутбуке я обучаю обычную полносвязную сеть, также использую кроссвалидацию

In [ ]:
import os
import pandas as pd
import numpy as np
import gc

import matplotlib.pyplot as plt
from scipy import stats
import torch

import torchvision
import torch.nn as nn

import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
gc.collect()

21

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv
/kaggle/input/ubiquant-market-prediction/example_test.csv
/kaggle/input/ubiquant-market-prediction/train.csv
/kaggle/input/ubiquant-market-prediction/ubiquant/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/ubiquant-market-prediction/ubiquant/__init__.py
/kaggle/input/ubiquant-market-prediction-half-precision-pickle/train.pkl


## Импортирум данные

In [ ]:
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
train.head(2)

,investment_id,time_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,...,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,target
0,1,0,0.932617,0.113708,-0.402100,0.378418,-0.203979,-0.413574,0.965820,1.230469,...,-1.095703,0.200073,0.819336,0.941406,-0.086792,-1.086914,-1.044922,-0.287598,0.321533,-0.300781
1,2,0,0.811035,-0.514160,0.742188,-0.616699,-0.194214,1.771484,1.427734,1.133789,...,0.912598,-0.734375,0.819336,0.941406,-0.387695,-1.086914,-0.929688,-0.974121,-0.343506,-0.231079


In [ ]:
inputs = train.drop(['target'], axis=1).values
targets = train[['target']].values

inputs.shape, targets.shape

((3141410, 302), (3141410, 1))

In [ ]:
val_1 = int(0.8*inputs.shape[0])
val_2 = int(0.2*inputs.shape[0])
val_1, val_2

(2513128, 628282)

### Гиперпараметры

In [ ]:
batch_size = 2000
TARGET_COLUMN = 'target'
input_size=302
output_size=1

In [ ]:
del train
gc.collect()

63

# Кастомный датасет прописывать не стал

In [ ]:
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32))
train_ds, val_ds = random_split(dataset, [val_1, val_2])

train_loader = DataLoader(train_ds, batch_size, shuffle=False)
val_loader = DataLoader(val_ds, batch_size*2)

In [ ]:
del train_ds, val_ds, dataset, inputs, targets
gc.collect()

42

## Перенос данных на ГПУ

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()
device

device(type='cuda')

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(train_loader, device)

# Создание модели

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(input_size, 64), 
                                    nn.BatchNorm1d(64), 
                                    nn.SiLU(), 
                                    
                                    nn.Linear(64, 128), 
                                    nn.BatchNorm1d(128),  
                                    nn.SiLU(),
                                    nn.Dropout(0.4),
                                    
                                    nn.Linear(128, 256), 
                                    nn.BatchNorm1d(256), 
                                    nn.SiLU(), 
                                    nn.Dropout(0.4),
                                    
                                    nn.Linear(256, 512), 
                                    nn.BatchNorm1d(512), 
                                    nn.SiLU(),
                                    nn.Dropout(0.4), 
                                    
                                    nn.Linear(512, 256), 
                                    nn.BatchNorm1d(256), 
                                    nn.SiLU(),
                                    nn.Dropout(0.4),
                                    
                                    nn.Linear(256, 128), 
                                    nn.BatchNorm1d(128), 
                                    nn.SiLU(),
                                    nn.Dropout(0.4),
                                    
                                    nn.Linear(128, 8), 
                                    nn.BatchNorm1d(8), 
                                    nn.SiLU(), 
                                    nn.Dropout(0.4),
                                    
                                    nn.Linear(8, 1) )
    
        
    def forward(self, x):
        return self.layers(x)
    
    def training_step(self, batch):
        torch.cuda.empty_cache()
        gc.collect()
        inputs, targets = batch 
        inputs.to(device)
        targets.to(device)
        
        out = self(inputs)                 
        loss = F.mse_loss(out, targets)   
        return loss
    
    def validation_step(self, batch):
        torch.cuda.empty_cache()
        gc.collect()
        inputs, targets = batch 
        inputs.to(device)
        targets.to(device)
        
        out = self(inputs)                 
        loss = F.mse_loss(out, targets)    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result['train_loss'], result['val_loss']))
    
model = Model()

In [ ]:
model = to_device(model, device)

### Обучение

In [ ]:
def evaluate(model, val_loader):
    model.eval()  
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs): 
        model.train() 
        train_losses = []
        
        for (i,batch) in enumerate(train_loader):
            torch.cuda.empty_cache()
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if (i%200 == 0): print('batch number -- ', i)
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        
            
    return history

In [ ]:
train_loader.device, val_loader.device

(device(type='cuda'), device(type='cuda'))

In [ ]:
model.state_dict()['layers.0.weight']

tensor([[-0.0450, -0.0543,  0.0315,  ..., -0.0190, -0.0150,  0.0005],
        [ 0.0494,  0.0416, -0.0041,  ..., -0.0243, -0.0446, -0.0407],
        [-0.0487, -0.0179,  0.0028,  ..., -0.0072,  0.0504, -0.0491],
        ...,
        [-0.0339,  0.0208, -0.0226,  ...,  0.0098,  0.0115, -0.0399],
        [ 0.0134,  0.0431, -0.0346,  ..., -0.0427,  0.0519,  0.0390],
        [-0.0512,  0.0464,  0.0565,  ..., -0.0246, -0.0021, -0.0568]],
       device='cuda:0')

In [ ]:
learning_rate = 1e-1

In [ ]:
gc.collect()

147

In [ ]:
history = fit(10, learning_rate, model, train_loader, val_loader, opt_func=torch.optim.Adam)

batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [0], train_loss: 0.8441, val_loss: 0.8521
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [1], train_loss: 0.8361, val_loss: 0.8391
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [2], train_loss: 0.8349, val_loss: 0.8434
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [3], train_loss: 0.8346, val_loss: 0.8323
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [4], train_loss: 0.8343, val_loss: 0.8360
batch number --  0
batch 

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');
history

[{'val_loss': 0.8521440029144287, 'train_loss': 0.8441387414932251},
 {'val_loss': 0.8390878438949585, 'train_loss': 0.8361467719078064},
 {'val_loss': 0.8434250354766846, 'train_loss': 0.8349288702011108},
 {'val_loss': 0.8322630524635315, 'train_loss': 0.8345692753791809},
 {'val_loss': 0.8360334634780884, 'train_loss': 0.8343096971511841},
 {'val_loss': 0.8320338726043701, 'train_loss': 0.8336008787155151},
 {'val_loss': 0.8304229378700256, 'train_loss': 0.8337907195091248},
 {'val_loss': 0.837978184223175, 'train_loss': 0.8333766460418701},
 {'val_loss': 0.8299148678779602, 'train_loss': 0.832933247089386},
 {'val_loss': 0.8339259028434753, 'train_loss': 0.8323901891708374}]

In [ ]:
history2 = fit(10, 5e-3, model, train_loader, val_loader, opt_func=torch.optim.Adam)

batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [0], train_loss: 0.8284, val_loss: 0.8256
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [1], train_loss: 0.8279, val_loss: 0.8256
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [2], train_loss: 0.8277, val_loss: 0.8254
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [3], train_loss: 0.8275, val_loss: 0.8252
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [4], train_loss: 0.8274, val_loss: 0.8250
batch number --  0
batch 

In [ ]:
history3 = fit(10, 5e-5, model, train_loader, val_loader, opt_func=torch.optim.Adam)

batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [0], train_loss: 0.8264, val_loss: 0.8239
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [1], train_loss: 0.8264, val_loss: 0.8238
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [2], train_loss: 0.8261, val_loss: 0.8238
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [3], train_loss: 0.8263, val_loss: 0.8238
batch number --  0
batch number --  200
batch number --  400
batch number --  600
batch number --  800
batch number --  1000
batch number --  1200
Epoch [4], train_loss: 0.8263, val_loss: 0.8238
batch number --  0
batch 

In [ ]:
history3[-1]

{'val_loss': 0.8237826228141785, 'train_loss': 0.8261511921882629}

In [ ]:
torch.save(model.state_dict(), 'my_trained_model.pth')

# Делаем предсказания

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gc.collect()

3257

In [ ]:
cols_order = ['investment_id' , 'time_id'] + features

In [ ]:
def predict_for_test_data(test_data):
    test_ds = TensorDataset(torch.tensor(test_data.values, dtype=torch.float32))
    submission_try = []
    
    for x in test_ds:
        model.eval()
        input_x = x[0].unsqueeze(0).cuda()
        pred = model(input_x)
        submission_try.append(pred)
        print("Prediction:", pred)
        
    submission_values = [float(i.detach()) for i in submission_try]
    return submission_values

In [ ]:
predict_for_test_data(test_df)